In [1]:
import tensorflow

In [5]:
def preprocess_image(self, image_path): """ Load and preprocess an image. """ image = load_img(image_path, target_size=self.image_size) image = img_to_array(image) / 255.0 # Normalize to [0,1] return image def preprocess_mask(self, mask_path): """ Load and preprocess a mask, ensuring it's binary. """ mask = load_img(mask_path, target_size=self.image_size, color_mode='grayscale') mask = img_to_array(mask) / 255.0 # Normalize to [0,1] return np.where(mask > 0.5, 1, 0) # Binarize

Class Distribution:
MEL: 1113.0 cases
NV: 6705.0 cases
BCC: 514.0 cases
AKIEC: 327.0 cases
BKL: 1099.0 cases
DF: 115.0 cases
VASC: 142.0 cases
Starting dataset augmentation...


RuntimeError: invalid shape for coordinate array

In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.5 MB 975.2 kB/s eta 0:00:12
    --------------------------------------- 0.2/11.5 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.5/11.5 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/11.5 MB 4.4 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.5 MB 7.3 MB/s eta 0:00:02
   ------------ --------------------------- 3.5/11.5 MB 11.9 MB/s eta 0:00:01
   ----------------------- ---------------- 6.7/11.5 MB 19.3 MB/s eta 0:00:01
   -------------------------------------- - 11.0/11.5 MB 54.7 MB/s eta 0:00:01
   ------------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install scipy


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 330.3 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.8 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 543.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/44.5 MB 3.2 MB/s eta 0:00:14
   ---------------------------------------- 0.2/44.5 MB 2.6 MB/s eta 0:00:18
   ---------------------------------------- 0.4/44.5 MB 3.4 MB/s eta 0:00:14
    --------------------------------------- 1.0/44.5 MB 5.5 MB/s eta 0:00:08
   - -------------------------------------- 1.8/44.5 MB 8.3 MB/s eta 0:00:06
   -- ------------------------------------- 2.8/44.5 MB 11.2 MB/s eta 0:00:04
   -- ------------------------------------- 2.8/44.5 MB 11.2 MB/s eta 0:00:04


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os 

import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image 
import load_img, img_to_array, save_img 
from scipy.ndimage import gaussian_filter, map_coordinates 

class SkinLesionDataLoader: def __init__(self, csv_path, image_dir, mask_dir, output_img_dir, output_mask_dir, augmented_csv_path, image_size=(128, 128), batch_size=32, seed=42): 
    """ Initialize the DataLoader with dataset paths and parameters. 
    Args: csv_path: Path to the CSV file with labels and metadata. 
    image_dir: Directory containing original images.
    mask_dir: Directory containing mask images.
    output_img_dir: Directory to save augmented images. 
    output_mask_dir: Directory to save augmented masks. 
    augmented_csv_path: Path to save the augmented dataset CSV. 
    image_size: Tuple of (height, width) for resizing images. 
    batch_size: Number of samples per batch. seed: Seed for reproducibility of augmentations. """ 
    self.csv_path = csv_path 
    self.image_dir = image_dir 
    self.mask_dir = mask_dir 
    self.output_img_dir = output_img_dir 
    self.output_mask_dir = output_mask_dir 
    self.augmented_csv_path = augmented_csv_path
     self.image_size = image_size 
    self.batch_size = batch_size 
    self.seed = seed 
    # Read the original CSV 
    self.df = pd.read_csv(csv_path) 
    # Define augmentation factors 
    self.augment_factors = { 'MEL': 4, 
                            'NV': 1, # Skip augmentation for NV 
                            'BCC': 4, 
                            'AKIEC': 9, 
                            'BKL': 7, 
                            'DF': 18, 
                            'VASC': 15 } # Create output directories 
    os.makedirs(self.output_img_dir, exist_ok=True) 
    os.makedirs(self.output_mask_dir, exist_ok=True) 
    
def preprocess_image(self, image_path):
    """ Load and preprocess an image. """ 
    image = load_img(image_path, target_size=self.image_size) 
    image = img_to_array(image) / 255.0 # Normalize to [0,1] 
    return image 

def preprocess_mask(self, mask_path): 
    """ Load and preprocess a mask, ensuring it's binary. """ 
    mask = load_img(mask_path, target_size=self.image_size, color_mode='grayscale') 
    mask = img_to_array(mask) / 255.0 
    # Normalize to [0,1] 
    return np.where(mask > 0.5, 1, 0) # Binarize 

def elastic_deformation(self, image, mask, alpha=34, sigma=4): 
    """ Apply elastic deformation to an image and its corresponding mask. 
    Args: image: The input image as a NumPy array.
    mask: The corresponding mask as a NumPy array. 
    alpha: Scaling factor for the intensity of deformation. 
    sigma: Standard deviation of the Gaussian filter. """ 
    random_state = np.random.RandomState(self.seed) # Generate random displacement fields 
    shape = image.shape[:2] 
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha 
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha 
    # Create meshgrid for pixel indices 
    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0])) 
    indices = np.reshape(y + dy, (-1, 1)), np.reshape(x + dx, (-1, 1))
    # Apply deformation to the image and mask 
    deformed_image = map_coordinates(image, indices, order=1, mode='reflect').reshape(image.shape) 
    deformed_mask = map_coordinates(mask, indices, order=1, mode='reflect').reshape(mask.shape) 
    return deformed_image, deformed_mask

def augment(self, image, mask): 
    """ Apply augmentations to an image and its corresponding mask. """ 
    tf.random.set_seed(self.seed) 
    augmentation_pipeline = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical", seed=self.seed), 
        tf.keras.layers.RandomRotation(0.15, seed=self.seed), 
        tf.keras.layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2), seed=self.seed), 
        tf.keras.layers.RandomContrast(0.2, seed=self.seed) 
        ]) 
    # Apply augmentations to both image and mask 
    augmented_image = augmentation_pipeline(image) 
    augmented_mask = augmentation_pipeline(mask) 
    # Ensure mask is binary 
    augmented_mask = tf.where(augmented_mask > 0.5, 1.0, 0.0) 
    # Apply CLAHE for contrast enhancement 
    augmented_image_uint8 = tf.cast(augmented_image * 255, tf.uint8).numpy() 
    lab = cv2.cvtColor(augmented_image_uint8, cv2.COLOR_RGB2LAB) 
    lab_planes = cv2.split(lab) 
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) 
    lab_planes[0] = clahe.apply(lab_planes[0]) 
    lab = cv2.merge(lab_planes) 
    augmented_image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB) / 255.0 
    # Add minimal noise 
    noise = tf.random.normal(shape=tf.shape(augmented_image), mean=0.0, stddev=0.02) 
    augmented_image = augmented_image + noise 
    augmented_image = tf.clip_by_value(augmented_image, 0.0, 1.0) 
    # Apply elastic deformation 
    augmented_image, augmented_mask = self.elastic_deformation(augmented_image, augmented_mask.numpy()) 
    return augmented_image, augmented_mask

def augment_dataset(self): 
    """ Augment the dataset, save augmented images and masks, and generate a new CSV file. """ 
    print("Starting dataset augmentation...") 
    augmented_data = [] # List to hold augmented dataset information 
    for idx, row in self.df.iterrows(): 
        image_id = row['image'] 
        image_path = os.path.join(self.image_dir, image_id + '.jpg') 
        mask_path = os.path.join(self.mask_dir, image_id + '_Segmentation.png') 
        # Check if image and mask exist 
        if not os.path.exists(image_path) or not os.path.exists(mask_path): 
            print(f"Warning: Missing files for {image_id}") 
            continue 
        # Load and preprocess 
        image = self.preprocess_image(image_path) 
        mask = self.preprocess_mask(mask_path) 
        # Determine the augmentation factor 
        augment_factor = max(self.augment_factors[col] for col in self.augment_factors if row[col] == 1) 
        for aug_idx in range(augment_factor): 
            # Apply augmentation 
            augmented_image, augmented_mask = self.augment( tf.expand_dims(image, 0), tf.expand_dims(mask, 0) ) 
            # Save augmented image and mask
            aug_image_id = f"{image_id}_aug_{aug_idx}" 
            aug_image_path = os.path.join(self.output_img_dir, aug_image_id + '.jpg') 
            aug_mask_path = os.path.join(self.output_mask_dir, aug_image_id + '.png') 
            save_img(aug_image_path, augmented_image * 255.0) # Convert to uint8 save_img(aug_mask_path, 
            augmented_mask * 255.0, scale=False) 
            # Append to augmented data 
            labels = [row[col] for col in ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']] 
            augmented_data.append([aug_image_id, aug_image_path, aug_mask_path] + labels) 
        # Create DataFrame for augmented data 
        augmented_df = pd.DataFrame(augmented_data, columns=['image', 'image_path', 'mask_path', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']) 
        # Save augmented dataset to CSV 
        augmented_df.to_csv(self.augmented_csv_path, index=False) 
        print(f"Augmentation completed. Augmented dataset saved to {self.augmented_csv_path}.")